# Analysis of DIC data from tensile testing

## Import libraries

In [11]:
import glob
from pathlib import Path
from typing import List, Dict

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from ipywidgets import interact, fixed, IntSlider, Dropdown, SelectMultiple, Checkbox
from dic_analysis.dic import DeformationMap
import dic_analysis.io

import utilities

# Download Data
The experimental data (~100 MB) is downloaded to the `data` directory in the root of the repository.

In [12]:
data_locations = utilities.read_data_yaml("../data/zenodo_URLs.yaml")["tensile_tests"]
print(data_locations)

data_folder = Path("../data/tensile_tests")

data_file = utilities.get_file_from_url(data_folder, **data_locations)
utilities.unzip_file(data_file, data_folder)
print(data_folder)

{'url': 'https://sandbox.zenodo.org/record/779690/files/tensile_tests.zip', 'md5': '26a03e24dcf4d055e0e9e79e0937036c', 'name': 'tensile_tests.zip'}
MD5 validated. Download complete.
Unzipping...complete.
../data/tensile_tests


## Analysis set up
Change the data location and number of files to load here.

In [13]:
sample_angles = ["0-1"]#, "0-2", "30-1", "30-2", "45-1", "45-2", "60-1", "60-2", "90-1", "90-2"]
# If max frame is None, it will load all maps from the data folder, if it is a number it will load that many.
max_frame = None

Load data from files

In [14]:
# Load data from files and put maps into a dictionary labelled by sample angle.
deformation_maps = {}

for angle in tqdm(sample_angles, desc="Sample angle"):
    angle_folder = data_folder / f"test {angle}/displacement data/"
    file_list = glob.glob(f"{angle_folder}/*")
    file_list = np.sort(file_list)
    if not max_frame:
        deformation_maps[angle] = [DeformationMap(file_path, [0, 1, 2, 3]) for file_path in tqdm(file_list, leave=False, desc='File number')]
    else:
        deformation_maps[angle] = [DeformationMap(file_list[frame_num], [0, 1, 2, 3]) for
                                   frame_num in tqdm(range(1, max_frame), leave=False, desc='File number')]

Sample angle:   0%|          | 0/1 [00:00<?, ?it/s]

File number:   0%|          | 0/223 [00:00<?, ?it/s]

In [15]:
# Set up folders for file output
RESULTS_DIR = Path('../results/tensile_test_DIC')
RESULTS_DIR.mkdir(exist_ok=True, parents=True)
for angle in sample_angles:
    directory = RESULTS_DIR / Path(f"{angle}")
    directory.mkdir(exist_ok=True)

## Plotting strain maps

This cell allows scanning through the true strain maps over time. This can be used to determine which timesteps are interesting to output.

In [16]:
file_widget = Dropdown(options=sample_angles)
timestep_widget = IntSlider(min=0, max=len(deformation_maps[sample_angles[0]]) - 1, step=1, continuous_update=False)

def scrub_strain(experiment_name: str, time_step: int, deformation_maps: dict):
    plt.imshow(np.log(deformation_maps[experiment_name][time_step].f22 + 1))
    plt.colorbar()
    plt.show()

# Dynamically update the maximum value of the timestep value dependent on the number of images in the experiment.
def update_timestep_range(*args):
    timestep_widget.max = len(deformation_maps[file_widget.value]) - 1
file_widget.observe(update_timestep_range, 'value')

interact(scrub_strain,
         experiment_name=file_widget,
         time_step=timestep_widget,
         deformation_maps=fixed(deformation_maps),
         continuous_update=False);

interactive(children=(Dropdown(description='experiment_name', options=('0-1',), value='0-1'), IntSlider(value=…

## Plotting shape change of sample

rho is the shape change -deyy/dexx

In [17]:
file_widget = Dropdown(options=sample_angles)
timestep_widget = IntSlider(min=1, max=len(deformation_maps[sample_angles[0]]) - 1, step=1, continuous_update=False)

def scrub_rho(experiment_name: int, time_step: str, deformation_maps: dict):
    rho = -deformation_maps[experiment_name][time_step].f11 / deformation_maps[experiment_name][
        time_step].f22
    plt.imshow(rho)
    plt.colorbar()
    plt.show()


# Dynamically update the maximum value of the timestep value dependent on the number of images in the experiment.
def update_timestep_range(*args):
    timestep_widget.max = len(deformation_maps[file_widget.value]) - 1
file_widget.observe(update_timestep_range, 'value')

interact(scrub_rho,
         experiment_name=file_widget,
         time_step=timestep_widget,
         deformation_maps=fixed(deformation_maps),
         continuous_update=False);

interactive(children=(Dropdown(description='experiment_name', options=('0-1',), value='0-1'), IntSlider(value=…

## Plotting sample strain/true strain over time

We crop the deformation map to select only the center of the sample by setting the x_range and y_range parameters. These select the pixel range used to calculate the strain.

In [18]:
x_range = (1, 12)
y_range = (10, 24)


def plot_strain(sample_angles: List[str], plot_eng_strain: bool, 
                mean_strain: Dict[str, np.ndarray],
               mean_true_strain: Dict[str, np.ndarray]):
    
    plt.figure(figsize=(10, 6))

    for angle in sample_angles:
        # Plot mean strain and mean true strain against time
        plt.plot(mean_true_strain[angle], label=f"true strain {angle}")
        if plot_eng_strain:
            plt.plot(mean_strain[angle], label=f"eng strain {angle}")

    plt.xlabel("Time step")
    plt.ylabel("Strain")
    plt.legend(bbox_to_anchor=(1, 1))
    plt.show()


# Mean strain over time, one for each sample angle
mean_strain = {}
mean_true_strain = {}

# Loop over all sample angles
for angle in sample_angles:
    mean_strain[angle] = []

    # Loop over all time steps
    for def_map in deformation_maps[angle]:
        # Crop the map the center and calculate the mean longitudinal strain
        cropped_map = def_map.f22[y_range[0]:y_range[1], x_range[0]:x_range[1]]
        mean_strain[angle].append(np.mean(cropped_map))
    # Convert list of mean strains to np array
    mean_strain[angle] = np.array(mean_strain[angle])
    # Compute true strains
    mean_true_strain[angle] = np.log(mean_strain[angle] + 1)
    
interact(plot_strain, sample_angles=SelectMultiple(options=sample_angles, value=sample_angles), 
         plot_eng_strain=Checkbox(), mean_strain=fixed(mean_strain), mean_true_strain=fixed(mean_true_strain));

interactive(children=(SelectMultiple(description='sample_angles', index=(0,), options=('0-1',), value=('0-1',)…

## Plotting transverse strain and longitudinal strain over time
Again we select only the ceter of the sample to calcualte the mean strain.

In [19]:
def plot_transverse_strain(sample_angles: List[str], plot_strain: bool, 
                           mean_strain: Dict[str, np.ndarray], mean_trans_strain: Dict[str, np.ndarray],
                          mean_true_trans_strain: Dict[str, np.ndarray]):

    plt.figure(figsize=(10, 6))
    for angle in sample_angles:
        if plot_strain:
            plt.plot(mean_true_strain[angle], label=f"true strain {angle}")
        plt.plot(mean_true_trans_strain[angle], label=f"true transverse_strain {angle}")

    plt.xlabel('Time step')
    plt.ylabel('Strain')
    plt.legend(bbox_to_anchor=(1, 1))
    plt.show()


# Mean transverse strain over time, one for each sample angle
mean_trans_strain = {}
mean_true_trans_strain = {}
    
# Loop over all sample angles
for angle in sample_angles:
    mean_trans_strain[angle] = []
    # Loop over all time steps
    for def_map in deformation_maps[angle]:
        # Crop the map to the center and calculate the mean transverse strain
        cropped_map = def_map.f11[y_range[0]:y_range[1], x_range[0]:x_range[1]]
        mean_trans_strain[angle].append(np.mean(cropped_map))
    # Convert list of mean transverse strains to np array
    mean_trans_strain[angle] = np.array(mean_trans_strain[angle])
    # Compute true transverse strain
    mean_true_trans_strain[angle] = np.log(np.array(mean_trans_strain[angle]) + 1)

    
interact(plot_transverse_strain, sample_angles=SelectMultiple(options=sample_angles, value=sample_angles), 
         plot_strain=Checkbox(), mean_strain=fixed(mean_strain), mean_trans_strain=fixed(mean_trans_strain),
        mean_true_trans_strain=fixed(mean_true_trans_strain));

interactive(children=(SelectMultiple(description='sample_angles', index=(0,), options=('0-1',), value=('0-1',)…

## Plotting Strain ratio

Here we plot the ratio of the longitudinal strain to transverse strain.

We crop the data at a max and min longitudinal strain to avoid noisy data points

In [20]:
min_strain = 0.02
max_strain = 0.29

def plot_strain_ratio(sample_angles: List[str], mean_true_strain: Dict[str, np.ndarray], 
                      mean_true_trans_strain: Dict[str, np.ndarray]):
    plt.figure(figsize=(10, 6))
    
    for angle in sample_angles:

        with np.errstate(invalid='ignore'):
            true_strain_ratio = - mean_true_trans_strain[angle] / mean_true_strain[angle]

        mask = np.logical_and(min_strain < mean_true_strain[angle], mean_true_strain[angle] < max_strain)
        plt.plot(mean_true_strain[angle][mask], true_strain_ratio[mask], label=angle)

    plt.xlabel("true strain")
    plt.ylabel("true strain ratio")
    plt.legend(bbox_to_anchor=(1, 1))
    plt.show()


interact(plot_strain_ratio, sample_angles=SelectMultiple(options=sample_angles, value=sample_angles), 
         mean_true_strain=fixed(mean_true_strain), mean_true_trans_strain=fixed(mean_true_trans_strain));

interactive(children=(SelectMultiple(description='sample_angles', index=(0,), options=('0-1',), value=('0-1',)…

## Plotting Lankford parameter
As above, we cut the data at a minimum and maximum strain to reduce noise.

In [21]:
def plot_lankford(sample_angles: List[str], mean_true_strain: Dict[str, np.ndarray], 
                      mean_true_trans_strain: Dict[str, np.ndarray]):
    plt.figure(figsize=(10, 6))

    for angle in sample_angles:

        with np.errstate(invalid='ignore'):
            true_strain_ratio = - mean_true_trans_strain[angle] / mean_true_strain[angle]
        lankford = true_strain_ratio / (1 - true_strain_ratio)

        mask = np.logical_and(min_strain < mean_true_strain[angle], mean_true_strain[angle] < max_strain)
        plt.plot(mean_true_strain[angle][mask], lankford[mask], label=angle)

    plt.legend(bbox_to_anchor=(1, 1))
    plt.xlabel("true strain")
    plt.ylabel("Lankford parameter")
    plt.show()


interact(plot_lankford, sample_angles=SelectMultiple(options=sample_angles, value=sample_angles), 
         mean_true_strain=fixed(mean_true_strain), mean_true_trans_strain=fixed(mean_true_trans_strain));

interactive(children=(SelectMultiple(description='sample_angles', index=(0,), options=('0-1',), value=('0-1',)…

## Plotting Measured strain data

In [22]:
def plot_measured_strain(sample_angles: List[str], voltage_data: Dict[str, np.ndarray]):

    plt.figure(figsize=(10, 6))

    for angle in sample_angles:
        plt.plot(voltage_data[angle][:, 0], voltage_data[angle][:, 1], label=angle)
        plt.xlabel("True Strain")
        plt.ylabel("True Stress (MPa)")
        plt.legend()
    plt.show()


cropped_voltage_data = {}
    
for angle in sample_angles:
    voltage_data = np.loadtxt(data_folder / f"test {angle}/voltage data/data_1.csv", delimiter=",", skiprows=2, usecols=(4, 15))

    # Cut off data when it begins dropping at the end of the experiment
    data_limit = voltage_data.shape[0]
    for i in range(0, data_limit - 50):
        if voltage_data[i, 1] > voltage_data[i + 50, 1]:
            data_limit = i + 50
            break
    cropped_voltage_data[angle] = voltage_data[:data_limit, :2]
    

interact(plot_measured_strain, sample_angles=SelectMultiple(options=sample_angles, value=sample_angles), 
         voltage_data=fixed(cropped_voltage_data));

interactive(children=(SelectMultiple(description='sample_angles', index=(0,), options=('0-1',), value=('0-1',)…